### Load imports and run the no LID case:

In [ ]:
# Run this block to run the SWMM model on the 6-subcatchment Wingohocking model BEFORE adding GSI
%reset
import yaml
from swmm_objects import *
from swmm_read_cso_time_series import *
from swmm_run_cso import *
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
collection = 'randomTrial5'
client = MongoClient(mongoServer)
swmmInpFile = "wingohocking_PWD.inp"  # initial SWMM .inp file
lidZeroFile = "PWD_wingo_Wakefield_list_zero.yaml"
################################################

db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database

thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection,0,[0,0,0,0,0,0,0])  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

### Loop ALL subcatchment LID numbers over a range (using np.random):

In [ ]:
# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from swmm_run_cso import *
from copy import deepcopy
import numpy as np
permList = []
for i in range(0,10):
    randomPerm=np.random.randint(200, size=7)
    randomPerm = randomPerm.tolist()
    permList.append(randomPerm)
print permList
for perm in permList:
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    totalAreaTreated = 0
    for i in range(0,7):
        lidNumber = perm[i] #number of LIDs in subcatchment
        subArea = newRunList[i]['SubcatArea'] #area of subcatchment
        newRunList[i]['Number'] = perm[i]  # random number of lids in each subcatchment
        totalAreaTreated += lidNumber*subArea #area treated by each LID, added together   
    print totalAreaTreated
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection,totalAreaTreated,perm)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"

### To display all of the runs from the database:

In [ ]:
runs = db[collection]
cursor = runs.find()
#\for run in cursor:
#    print run
#    print ''

### To Reconstitute the SWMM input file for any of the runs above

In [ ]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
#print swmmInputFileStr
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()

# Now, apply a range of Wakefield Park rain gardens in all seven subcatchments simultaneously
from copy import deepcopy
for numberForAllLid in range(0,21):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newVolume = cursor['volume']
    volumeReduction = zeroLidVolume - newVolume
    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"